# Understanding Hired Rides in NYC

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In this part, we import all the libraries we will use in this project and set some constants to represent the dataset.

In [1]:
# all import statements needed for the project
from math import radians,sin,cos,asin,sqrt
import bs4 
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlite3
import sqlalchemy as db
from sqlalchemy import inspect
import typing
import pytest
import ipytest
import re
import csv
from pandas.testing import assert_frame_equal
import os.path
from __future__ import annotations

In [2]:
# Set constants pointing to dataset
TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
UBER_CSV = "uber_rides_sample.csv"
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

### Calculate distance and Add it to the dataset
Step 1: Calculate the distance between two coordinates using the haversine formula:  
Reference from Wikipedia  
![image](https://miro.medium.com/max/1392/0*mie5h2yduk6NmIym)  
Step 2: Add the distance  to the dataset as the new column

In [3]:
def calculate_distance(from_coord:tuple[float,float], to_coord:tuple[float,float]) -> float:
    #Convert decimal number to radian number
    from_lon,from_lat,to_lon,to_lat=map(radians,[from_coord[0],from_coord[1],to_coord[0],to_coord[1]])
    
    #using haversin formula
    dis_lon=to_lon-from_lon
    dis_lat=to_lat-from_lat
    haversine=sin(dis_lat/2)**2+cos(from_lat)*cos(to_lat)*sin(dis_lon/2)**2
    #6371 is the earth's radius(km)
    distance=6371*2*asin(sqrt(haversine))
    #round the result to two decimals
    return round(distance,2)

In [4]:
#TEST FOR calculate_distance()
#TEST BEGINS
@pytest.mark.parametrize(
    "from_coord,to_coord,expected",
    (
        ((-73.99981689,40.73835373),(-73.99951172,40.72321701),1.68),
        ((-73.994355,40.728225),(-73.99471,40.750325),2.46),
        ((-73.78808,40.642187),(-73.865042,40.725997),11.36),
        ((-73.992122,40.748577),(-73.806072,40.665272),18.21),
        ((-73.982321,40.768729),(-73.921654,40.660043),13.12),
    )
)

def test_calculate_distance(from_coord:tuple[float,float],to_coord,expected:tuple[float,float]) -> None:
    assert expected==calculate_distance(from_coord,to_coord)

ipytest.run("-vv", "-k", "test_calculate_distance")
#TEST ENDS

======================================= test session starts =======================================
platform win32 -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- C:\Program Files\Python310\python.exe
cachedir: .pytest_cache
rootdir: e:\projects\Python Final Project
collecting ... collected 5 items

tmpd118i4kp.py::test_calculate_distance[from_coord0-to_coord0-1.68] <- C:\Users\Lenovo\AppData\Local\Temp\ipykernel_48652\1071625851.py PASSED [ 20%]
tmpd118i4kp.py::test_calculate_distance[from_coord1-to_coord1-2.46] <- C:\Users\Lenovo\AppData\Local\Temp\ipykernel_48652\1071625851.py PASSED [ 40%]
tmpd118i4kp.py::test_calculate_distance[from_coord2-to_coord2-11.36] <- C:\Users\Lenovo\AppData\Local\Temp\ipykernel_48652\1071625851.py PASSED [ 60%]
tmpd118i4kp.py::test_calculate_distance[from_coord3-to_coord3-18.21] <- C:\Users\Lenovo\AppData\Local\Temp\ipykernel_48652\1071625851.py PASSED [ 80%]
tmpd118i4kp.py::test_calculate_distance[from_coord4-to_coord4-13.12] <- C:\Users\Lenovo\AppData\L

<ExitCode.OK: 0>

In [5]:
#Add distance to the dataset
def add_distance_column(df:pd.DataFrame) -> None:
    distance=[]
    for i,row in df[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']].iterrows():
        distance.append(calculate_distance((row['pickup_longitude'],row['pickup_latitude']),(row['dropoff_longitude'],row['dropoff_latitude'])))
    df['trip_distance']=distance
    #add distance to the new column 'trip_distance'

In [6]:
# TEST FOR add_distance_column
# TEST BEGINS
test_data = pd.DataFrame(
    {
        "pickup_longitude": [-73.99981689, -73.994355],
        "pickup_latitude": [40.73835373, 40.728225],
        "dropoff_longitude": [-73.99951172, -73.99471],
        "dropoff_latitude": [40.72321701, 40.750325],
    }
)
expected_data = pd.DataFrame(
    {
        "pickup_longitude": [-73.99981689, -73.994355],
        "pickup_latitude": [40.73835373, 40.728225],
        "dropoff_longitude": [-73.99951172, -73.99471],
        "dropoff_latitude": [40.72321701, 40.750325],
        "trip_distance": [1.68, 2.46],
    }
)


def test_add_distance() -> None:
    add_distance_column(test_data)
    assert_frame_equal(test_data, expected_data)


ipytest.run("-vv", "-k", "test_add_distance")
# TEST ENDS


======================================= test session starts =======================================
platform win32 -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- C:\Program Files\Python310\python.exe
cachedir: .pytest_cache
rootdir: e:\projects\Python Final Project
collecting ... collected 6 items / 5 deselected / 1 selected

tmpcrw6d52f.py::test_add_distance <- C:\Users\Lenovo\AppData\Local\Temp\ipykernel_48652\58012485.py PASSED [100%]

================================= 1 passed, 5 deselected in 0.02s =================================


<ExitCode.OK: 0>

### Processing Uber Data

* Clean the data, including:
  * Remove unnecessary columns
  * Remove invalid data points
  * Remove trips that start and/or end outside the designated NEW_YORK_BOX_COORDS  
  

* Calculate the sample size for each month and so we can use it later to sample the Yellow taxi dataset

In [7]:
# load and clean data
def load_and_clean_uber_data(csv_file:str) -> pd.DataFrame:
    df = pd.read_csv(csv_file)
    # only pick the columns that will be used later
    df=df[['fare_amount','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']]
    # Remove trips that start and/or end outside the designated coordinate box
    df = df[
        (df["pickup_longitude"] <= NEW_YORK_BOX_COORDS[1][1])
        & (NEW_YORK_BOX_COORDS[0][1] <= df["pickup_longitude"])
        & (df["pickup_latitude"] <= NEW_YORK_BOX_COORDS[1][0])
        & (NEW_YORK_BOX_COORDS[0][0] <= df["pickup_latitude"])
        & (df["dropoff_longitude"] <= NEW_YORK_BOX_COORDS[1][1])
        & (NEW_YORK_BOX_COORDS[0][1] <= df["dropoff_longitude"])
        & (df["dropoff_latitude"] <= NEW_YORK_BOX_COORDS[1][0])
        & (NEW_YORK_BOX_COORDS[0][0] <= df["dropoff_latitude"])
    ]
    # delete invalid data points where the fare_amount is 0
    return df[(df["fare_amount"] != 0)]

# get the whole cleaned dataset
def get_uber_data() -> pd.DataFrame:
    # get the cleaned data and add trip_distance column to it
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    return uber_dataframe


In [8]:
# calculate the sample size for each month
def get_sample_size(df:pd.DataFrame) -> dict:
    #group the data by the month
    df_month=df.groupby(df['pickup_datetime'].str[:7]).count()
    sample_size={}
    #store the month and sample size to the dict
    for month, size in df_month[['fare_amount']].iterrows():
        sample_size[month] = size.values[0]
    return sample_size

In [9]:
# run all functions and get the dataset
uber_data = get_uber_data()
sample_size=get_sample_size(uber_data)

In [10]:
# show the uber dataset
uber_data

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance
0,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1.68
1,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,2.46
2,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,5.04
3,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,1.66
4,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,4.48
...,...,...,...,...,...,...,...
199995,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,0.11
199996,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1.88
199997,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,12.85
199998,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,3.54


In [11]:
# show the sample size
print(sample_size)

{'2009-01': 2504, '2009-02': 2261, '2009-03': 2665, '2009-04': 2531, '2009-05': 2619, '2009-06': 2495, '2009-07': 2454, '2009-08': 2386, '2009-09': 2448, '2009-10': 2606, '2009-11': 2479, '2009-12': 2629, '2010-01': 2618, '2010-02': 1909, '2010-03': 2337, '2010-04': 2690, '2010-05': 2739, '2010-06': 2544, '2010-07': 2536, '2010-08': 2204, '2010-09': 2422, '2010-10': 2573, '2010-11': 2455, '2010-12': 2466, '2011-01': 2346, '2011-02': 2511, '2011-03': 2768, '2011-04': 2607, '2011-05': 2681, '2011-06': 2738, '2011-07': 2515, '2011-08': 2340, '2011-09': 2577, '2011-10': 2797, '2011-11': 2623, '2011-12': 2599, '2012-01': 2592, '2012-02': 2642, '2012-03': 2822, '2012-04': 2740, '2012-05': 2722, '2012-06': 2524, '2012-07': 2516, '2012-08': 2548, '2012-09': 2611, '2012-10': 2617, '2012-11': 2525, '2012-12': 2672, '2013-01': 2598, '2013-02': 2417, '2013-03': 2739, '2013-04': 2677, '2013-05': 2547, '2013-06': 2558, '2013-07': 2455, '2013-08': 2237, '2013-09': 2547, '2013-10': 2668, '2013-11': 25

### Processing Taxi Data

* use regular expression to find all the taxi csv file urls
* Use `requests`, BeautifulSoup (`bs4`), and `pandas` to programmatically download the required CSV files & load into memory.
* Clean the data, including:
    * Remove unnecessary columns
    * Remove invalid data points (take a moment to consider what's invalid)
    * Normalize column names
    * Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
* Sample the data so that we have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* Cache the sampling by saving it as a CSV file before moving onto the next file. 

In [12]:
# find all the taxi csv file urls
def find_taxi_csv_urls() -> list:
    taxi_page = requests.get(TAXI_URL)
    soup = bs4.BeautifulSoup(taxi_page.text, "html.parser")
    # create a pattern matching the taxi data csv files.
    pattern = re.compile(
        "https:\/\/s3\.amazonaws\.com\/nyc-tlc\/trip\+data\/yellow_tripdata_\d{4}-\d{2}\.csv"
    )
    # get all urls
    all_href = [a["href"] for a in soup.find_all("a")]
    # match the pattern with all urls and find out csv files urls
    all_url = pattern.findall("".join(all_href))
    # only select urls from 2009-01 to 2015-06
    all_url = all_url[54:60] + all_url[66:]
    return all_url


# Normalize all csv files col names to the same column names
def normalize_col_name(df: pd.DataFrame) -> pd.DataFrame:
    # get the original col names
    col_names = df.columns
    col_names = ",".join(col_names).split(",")
    # Set all the names to the following new names
    col_newname = [
        "pickup_datetime",
        "dropoff_datetime",
        "pickup_longitude",
        "pickup_latitude",
        "dropoff_longitude",
        "dropoff_latitude",
        "tip_amount",
        "total_amount",
    ]
    col_oldname = []
    # find all the old col needed to be changed
    pattern1 = ".*[Tt]ime.*"
    pattern2 = ".*[Ll]on.*"
    pattern3 = ".*[Ll]at.*"
    pattern4 = ".*[Tt]ip.*"
    pattern5 = ".*[Tt]otal.*"
    for i in col_names:
        if re.findall(pattern1, i) != []:
            col_oldname.append(re.findall(pattern1, i)[0])
        if re.findall(pattern2, i) != []:
            col_oldname.append(re.findall(pattern2, i)[0])
        if re.findall(pattern3, i) != []:
            col_oldname.append(re.findall(pattern3, i)[0])
        if re.findall(pattern4, i) != []:
            col_oldname.append(re.findall(pattern4, i)[0])
        if re.findall(pattern5, i) != []:
            col_oldname.append(re.findall(pattern5, i)[0])
    # assign old col names and new col names to a dict
    name_dict = dict(zip(col_oldname, col_newname))
    # change col names
    df.rename(columns=name_dict, inplace=True)
    # We only select part of the columns and remove unnecessary columns
    return df[col_newname]


# load and clean data
# if the csv files has been saved before, then load it directly from the computer
def get_and_clean_month_taxi_data(url: str) -> pd.DataFrame:
    # get the month from url
    month = url[59:66]
    taxi_sample = sample_size[month]
    file_exists = os.path.exists(f"taxi_{month}.csv")
    # if file already saved, then load it from the computer
    if file_exists == True:
        df = pd.read_csv(f"taxi_{month}.csv")
    # if file haven't been saved, then load it from the URL.
    else:
        df = pd.read_csv(url,on_bad_lines='skip')
        # Normalize the col names and delete other unnecessary columns
        df = normalize_col_name(df)
        # delete invalid data points where total_amount is zero.
        df = df[(df["total_amount"] != 0)]
        # remove trips that start or/and end out of NEW_YORK_BOX_COORDS
        df = df[
            (df["pickup_longitude"] <= NEW_YORK_BOX_COORDS[1][1])
            & (NEW_YORK_BOX_COORDS[0][1] <= df["pickup_longitude"])
            & (df["pickup_latitude"] <= NEW_YORK_BOX_COORDS[1][0])
            & (NEW_YORK_BOX_COORDS[0][0] <= df["pickup_latitude"])
            & (df["dropoff_longitude"] <= NEW_YORK_BOX_COORDS[1][1])
            & (NEW_YORK_BOX_COORDS[0][1] <= df["dropoff_longitude"])
            & (df["dropoff_latitude"] <= NEW_YORK_BOX_COORDS[1][0])
            & (NEW_YORK_BOX_COORDS[0][0] <= df["dropoff_latitude"])
        ]
        # Sample the data so that we have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
        df = df.sample(n=taxi_sample, random_state=1)
        df.to_csv(f"taxi_{month}.csv")
    return df

# get the whole cleaned taxi data
def get_and_clean_taxi_data() ->pd.DataFrame:
    all_taxi_dataframes = []
    all_csv_urls = find_taxi_csv_urls()
    for csv_url in all_csv_urls:
        dataframe = get_and_clean_month_taxi_data(csv_url)
        # Add trip_distance column to the dataset
        add_distance_column(dataframe)
        all_taxi_dataframes.append(dataframe)
    # concat all the dataframes together to one dataframe
    taxi_data = pd.concat(all_taxi_dataframes)
    taxi_data = taxi_data[
    [
        "pickup_datetime",
        "dropoff_datetime",
        "pickup_longitude",
        "pickup_latitude",
        "dropoff_longitude",
        "dropoff_latitude",
        "tip_amount",
        "total_amount",
        "trip_distance",
    ]
]
    return taxi_data


In [13]:
# run all the functions and get the dataset
taxi_data = get_and_clean_taxi_data()

In [14]:
# show taxi data
taxi_data

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_amount,total_amount,trip_distance
0,2015-01-13 07:03:21,2015-01-13 07:07:44,-73.961639,40.811272,-73.972099,40.794521,0.00,6.80,2.06
1,2015-01-22 04:24:33,2015-01-22 04:32:29,-73.967201,40.765133,-73.967201,40.765133,0.00,9.80,0.00
2,2015-01-11 02:13:11,2015-01-11 02:21:22,-73.993866,40.756718,-74.000473,40.740696,0.00,8.80,1.87
3,2015-01-31 23:07:04,2015-01-31 23:18:06,-73.996567,40.745354,-73.999924,40.725662,0.00,10.80,2.21
4,2015-01-28 12:19:09,2015-01-28 12:25:33,-73.955078,40.783367,-73.954170,40.774845,1.36,8.16,0.95
...,...,...,...,...,...,...,...,...,...
2624,2009-12-24 15:22:00,2009-12-24 15:30:00,-73.989560,40.747138,-73.975782,40.765718,0.00,7.00,2.37
2625,2009-12-21 12:15:40,2009-12-21 12:21:11,-73.936727,40.653765,-73.939208,40.652818,0.93,7.13,0.23
2626,2009-12-28 13:14:00,2009-12-28 13:17:00,-73.968078,40.762090,-73.960922,40.769287,0.00,4.60,1.00
2627,2009-12-12 14:33:19,2009-12-12 14:45:49,-73.978040,40.766393,-74.017330,40.711000,0.00,16.20,6.99


### Processing Weather Data

* Remove unnecessary columns that won't be used later
* Fill NaN values with zero because NaN means observe nothing
* Get the hourly and daily weather data
* Daily weather data is about all the daily mean of HourlyPrecipitation and HourlyWindSpeed



In [15]:
# Clean weather data and delete unnecessary columns
def clean_month_weather_data_hourly(csv_file: str) -> pd.DataFrame:
    df=pd.read_csv(csv_file,low_memory=False)
    # only get columns we need later
    df=df[['DATE','HourlyPrecipitation','HourlyWindSpeed']]
    # fill NaN values with zero because NaN means there is no precipitation.
    df.fillna(0.00,inplace=True)
    # T means very trace precipitation, to simplify, we set it as zero.
    df['HourlyPrecipitation'].replace({'T':0.00},inplace=True)
    # convert the datatype to float
    df['HourlyPrecipitation']=pd.to_numeric(df['HourlyPrecipitation'],errors='coerce')
    # We only collect data to Jun 2015
    if(csv_file[:4]=='2015'):
        df=df.iloc[:5826]
    return df

# Group weather data by the same date and calculate the daily mean of HourlyPrecipitation and HourlyWindSpeed
def clean_month_weather_data_daily(csv_file: str) -> pd.DataFrame:
    df=clean_month_weather_data_hourly(csv_file)
    df=df.groupby(df['DATE'].str[:10]).mean()
    return df

# load and clean data
def load_and_clean_weather_data() -> pd.DataFrame:
    hourly_dataframes = []
    daily_dataframes = []

    weather_csv_files = ["2009_weather.csv","2010_weather.csv","2011_weather.csv","2012_weather.csv","2013_weather.csv","2014_weather.csv","2015_weather.csv"]

    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)

    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    return hourly_data, daily_data


In [16]:
# run all the function and get the dataset
hourly_weather, daily_weather = load_and_clean_weather_data()

In [17]:
# show the hourly weather dataset
hourly_weather

,DATE,HourlyPrecipitation,HourlyWindSpeed
0,2009-01-01T00:51:00,0.00,18.0
1,2009-01-01T01:51:00,0.00,18.0
2,2009-01-01T02:51:00,0.00,18.0
3,2009-01-01T03:51:00,0.00,8.0
4,2009-01-01T04:51:00,0.00,11.0
...,...,...,...
5821,2015-06-30T23:43:00,0.04,7.0
5822,2015-06-30T23:49:00,0.00,5.0
5823,2015-06-30T23:51:00,0.04,6.0
5824,2015-06-30T23:59:00,0.00,0.0


In [18]:
# show the daily weather dataset
daily_weather

,HourlyPrecipitation,HourlyWindSpeed
DATE,,
2009-01-01,0.000000,11.041667
2009-01-02,0.000000,6.593750
2009-01-03,0.000000,9.875000
2009-01-04,0.000000,7.370370
2009-01-05,0.000000,6.925926
...,...,...
2015-06-26,0.000000,4.400000
2015-06-27,0.062093,7.697674
2015-06-28,0.012766,4.872340


## Part 2: Storing Cleaned Data

* Delete database if exists before
* Create a new database
* Using the schema to create four tables
* Add dataframes to the database
* Show all tables

In [19]:
# check if the database already exists
file_exists = os.path.exists("project.db")
# if the database exists, then delete it because we need create a totally new database
if(file_exists==True):
    os.remove("project.db")
# create a new SQLite database named project
engine = db.create_engine(DATABASE_URL,echo=True)
# use schema.sql to create four tables schema
!sqlite3 project.db < schema.sql

In [20]:
# Add four dataframes to the database
# Don't run this cell multiple times because it will append the same dataframe again and again.
uber_data.to_sql('uber_trips',con=engine,if_exists='append',index=False)
taxi_data.to_sql('taxi_trips',con=engine,if_exists='append',index=False)
hourly_weather.to_sql('hourly_weather',con=engine,if_exists='append',index=False)
daily_weather.to_sql('daily_weather',con=engine,if_exists='append')

2022-04-25 14:11:55,431 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("uber_trips")
2022-04-25 14:11:55,439 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-25 14:11:55,511 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-25 14:11:56,912 INFO sqlalchemy.engine.Engine INSERT INTO uber_trips (fare_amount, pickup_datetime, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, trip_distance) VALUES (?, ?, ?, ?, ?, ?, ?)
2022-04-25 14:11:56,912 INFO sqlalchemy.engine.Engine [generated in 1.18939s] ((7.5, '2015-05-07 19:52:06 UTC', -73.99981689453125, 40.73835372924805, -73.99951171875, 40.72321701049805, 1.68), (7.7, '2009-07-17 20:04:56 UTC', -73.994355, 40.728225, -73.99471, 40.750325, 2.46), (12.9, '2009-08-24 21:45:00 UTC', -74.005043, 40.74077, -73.962565, 40.772647, 5.04), (5.3, '2009-06-26 08:22:21 UTC', -73.976124, 40.790844, -73.965316, 40.803349, 1.66), (16.0, '2014-08-28 17:47:00 UTC', -73.925023, 40.744085, -73.97308199999999, 40.761247, 4.48), (

2367

In [21]:
# show uber_trips table
engine.execute("SELECT * FROM uber_trips").fetchall()

2022-04-25 14:12:01,031 INFO sqlalchemy.engine.Engine SELECT * FROM uber_trips
2022-04-25 14:12:01,036 INFO sqlalchemy.engine.Engine [raw sql] ()


[(1, 7.5, '2015-05-07 19:52:06 UTC', -73.99981689453125, 40.73835372924805, -73.99951171875, 40.72321701049805, 1.68),
 (2, 7.7, '2009-07-17 20:04:56 UTC', -73.994355, 40.728225, -73.99471, 40.750325, 2.46),
 (3, 12.9, '2009-08-24 21:45:00 UTC', -74.005043, 40.74077, -73.962565, 40.772647, 5.04),
 (4, 5.3, '2009-06-26 08:22:21 UTC', -73.976124, 40.790844, -73.965316, 40.803349, 1.66),
 (5, 16.0, '2014-08-28 17:47:00 UTC', -73.925023, 40.744085, -73.97308199999999, 40.761247, 4.48),
 (6, 4.9, '2011-02-12 02:27:09 UTC', -73.96901899999999, 40.75591, -73.96901899999999, 40.75591, 0.0),
 (7, 24.5, '2014-10-12 07:04:00 UTC', -73.96144699999999, 40.693965000000006, -73.871195, 40.774297, 11.73),
 (8, 9.7, '2012-02-17 09:32:00 UTC', -73.975187, 40.745767, -74.00272, 40.743537, 2.33),
 (9, 12.5, '2012-03-29 19:06:00 UTC', -74.001065, 40.741787, -73.96304, 40.775012, 4.89),
 (10, 6.5, '2015-05-22 17:32:27 UTC', -73.9743881225586, 40.74695205688477, -73.98858642578125, 40.729804992675774, 2.25),

In [22]:
# show taxi_trips table
engine.execute("SELECT * FROM taxi_trips").fetchall()

2022-04-25 14:12:13,591 INFO sqlalchemy.engine.Engine SELECT * FROM taxi_trips
2022-04-25 14:12:13,592 INFO sqlalchemy.engine.Engine [raw sql] ()


[(1, '2015-01-13 07:03:21', '2015-01-13 07:07:44', -73.96163940429686, 40.81127166748047, -73.97209930419923, 40.79452133178711, 0.0, 6.8, 2.06),
 (2, '2015-01-22 04:24:33', '2015-01-22 04:32:29', -73.96720123291014, 40.765132904052734, -73.96720123291014, 40.765132904052734, 0.0, 9.8, 0.0),
 (3, '2015-01-11 02:13:11', '2015-01-11 02:21:22', -73.99386596679686, 40.756717681884766, -74.00047302246092, 40.74069595336913, 0.0, 8.8, 1.87),
 (4, '2015-01-31 23:07:04', '2015-01-31 23:18:06', -73.99656677246092, 40.74535369873047, -73.99992370605469, 40.72566223144531, 0.0, 10.8, 2.21),
 (5, '2015-01-28 12:19:09', '2015-01-28 12:25:33', -73.955078125, 40.783367156982415, -73.95417022705078, 40.774845123291016, 1.36, 8.16, 0.95),
 (6, '2015-01-08 20:32:12', '2015-01-08 20:39:58', -73.97823333740233, 40.7543830871582, -73.98754119873048, 40.73903274536133, 1.65, 9.95, 1.88),
 (7, '2015-01-16 07:23:28', '2015-01-16 07:33:42', -73.95301818847656, 40.77254867553711, -73.97560882568358, 40.75502777

In [23]:
# show hourly_weather table
engine.execute("SELECT * FROM hourly_weather").fetchall()

2022-04-25 14:12:25,426 INFO sqlalchemy.engine.Engine SELECT * FROM hourly_weather
2022-04-25 14:12:25,434 INFO sqlalchemy.engine.Engine [raw sql] ()


[(1, '2009-01-01T00:51:00', 0.0, 18.0),
 (2, '2009-01-01T01:51:00', 0.0, 18.0),
 (3, '2009-01-01T02:51:00', 0.0, 18.0),
 (4, '2009-01-01T03:51:00', 0.0, 8.0),
 (5, '2009-01-01T04:51:00', 0.0, 11.0),
 (6, '2009-01-01T05:51:00', 0.0, 18.0),
 (7, '2009-01-01T06:51:00', 0.0, 14.0),
 (8, '2009-01-01T07:51:00', 0.0, 8.0),
 (9, '2009-01-01T08:51:00', 0.0, 15.0),
 (10, '2009-01-01T09:51:00', 0.0, 8.0),
 (11, '2009-01-01T10:51:00', 0.0, 14.0),
 (12, '2009-01-01T11:51:00', 0.0, 14.0),
 (13, '2009-01-01T12:51:00', 0.0, 11.0),
 (14, '2009-01-01T13:51:00', 0.0, 14.0),
 (15, '2009-01-01T14:51:00', 0.0, 14.0),
 (16, '2009-01-01T15:51:00', 0.0, 5.0),
 (17, '2009-01-01T16:51:00', 0.0, 8.0),
 (18, '2009-01-01T17:51:00', 0.0, 5.0),
 (19, '2009-01-01T18:51:00', 0.0, 10.0),
 (20, '2009-01-01T19:51:00', 0.0, 11.0),
 (21, '2009-01-01T20:51:00', 0.0, 8.0),
 (22, '2009-01-01T21:51:00', 0.0, 7.0),
 (23, '2009-01-01T22:51:00', 0.0, 5.0),
 (24, '2009-01-01T23:51:00', 0.0, 3.0),
 (25, '2009-01-02T00:51:00', 0.0, 3

In [24]:
# show daily_weather table
engine.execute("SELECT * FROM daily_weather").fetchall()

2022-04-25 14:12:33,296 INFO sqlalchemy.engine.Engine SELECT * FROM daily_weather
2022-04-25 14:12:33,299 INFO sqlalchemy.engine.Engine [raw sql] ()


[(1, '2009-01-01', 0.0, 11.041666666666666),
 (2, '2009-01-02', 0.0, 6.59375),
 (3, '2009-01-03', 0.0, 9.875),
 (4, '2009-01-04', 0.0, 7.37037037037037),
 (5, '2009-01-05', 0.0, 6.925925925925926),
 (6, '2009-01-06', 0.004, 6.9),
 (7, '2009-01-07', 0.04666666666666667, 9.58974358974359),
 (8, '2009-01-08', 0.0, 11.192307692307692),
 (9, '2009-01-09', 0.0, 9.583333333333334),
 (10, '2009-01-10', 0.007735849056603774, 7.69811320754717),
 (11, '2009-01-11', 0.013421052631578948, 7.0),
 (12, '2009-01-12', 0.0, 6.217391304347826),
 (13, '2009-01-13', 0.0, 4.896551724137931),
 (14, '2009-01-14', 0.0, 8.76923076923077),
 (15, '2009-01-15', 0.002380952380952381, 7.380952380952381),
 (16, '2009-01-16', 0.0, 8.833333333333334),
 (17, '2009-01-17', 0.4996, 5.44),
 (18, '2009-01-18', 0.12272727272727274, 5.431818181818182),
 (19, '2009-01-19', 0.0, 2.673076923076923),
 (20, '2009-01-20', 0.0, 6.0),
 (21, '2009-01-21', 0.0, 8.291666666666666),
 (22, '2009-01-22', 0.0, 6.75),
 (23, '2009-01-23', 0.0

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [26]:
def write_query_to_file(query, outfile):
    raise NotImplemented()


### Query N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [27]:
QUERY_N = """
TODO
"""


In [28]:
engine.execute(QUERY_N).fetchall()


2022-04-25 14:07:02,050 INFO sqlalchemy.engine.Engine 
TODO

2022-04-25 14:07:02,050 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-25 14:07:02,050 INFO sqlalchemy.engine.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) near "TODO": syntax error
[SQL: 
TODO
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")


## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.
* [ ] Come up with 3 questions on your own that can be answered based on the data in the 4 tables. Create at least one visualization to answer each question. At least one visualization should require data from at least 3 tables.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_You don't have to query the data directly from the database. You can just re-use the pandas DataFrame that you created in Part 1._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))

    values = "..."  # use the dataframe to pull out values needed to plot

    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")

    plt.show()


In [ ]:
plot_visual_n(some_dataframe)
